# Goal

Try out the capabilities of the [DMPTool API](https://github.com/CDLUC3/dmptool/wiki/API).

I created a private DMP with id 

In [ ]:
import requests
import urlparse

class DMPTool(object):
    def __init__(self, token, prod=True):
        self.token = token
        self.prod = prod
        if prod:
            self.base_url = "https://dmptool.org/api/v1/"
        else:
            self.base_url = "https://dmp2-staging.cdlib.org/api/v1/"
        self.headers = {'Authorization': 'Token token={}'.format(self.token)}
                
    def get_url(self, path, headers=None):
        if headers is None:
            headers = self.headers
            
        url = self.base_url + path
        return requests.get(url, headers=headers)
    
    def plans(self, id_=None):
        """
        https://dmptool.org/api/v1/plans
        https://dmptool.org/api/v1/plans/:id
        """
        
        if id_ is None:
            return self.get_url("plans").json()
        else:
            return self.get_url("plans/{}".format(id_)).json()
                    
    def plans_full(self, id_=None, format_='json'):
    
        if id_ is None:
            # a json doc for to represent all public docs
            # I **think** if we include token, will get only docs owned
            return self.get_url("plans_full/", headers={}).json()
        else:
            if format_ == 'json':
                return self.get_url("plans_full/{}".format(id_)).json()
            elif format_ in ['pdf', 'docx']:
                return self.get_url("plans_full/{}.{}".format(id_, format_)).content
            else: 
                return None

    def plans_owned(self):
        return self.get_url("plans_owned").json()
    
    def plans_owned_full(self):
        return self.get_url("plans_owned_full").json()
    
    def plans_templates(self):
        return self.get_url("plans_templates").json()
        
    def institutions_plans_count(self):
        """
        https://github.com/CDLUC3/dmptool/wiki/API#for-a-list-of-institutions-and-plans-count
        """
        plans_counts = self.get_url("institutions_plans_count").json()
        return plans_counts

    

# How to get dmptool API token

![How to get dmptool token](https://www.evernote.com/shard/s1/sh/e8ff5420-33ef-4ef1-b77f-99beb4297481/cb6ffa697576c241/res/f2acb1a1-5b54-4738-8887-89788452d75c/skitch.png?resizeSmall&width=832)



In [ ]:
import requests

import numpy as np
import pandas as pd
from pandas import (DataFrame, Series)

from settings import (DMPTOOL_TOKEN, STAGING_DMPTOOL_TOKEN)

# https://dmptool.org/plans/16453.pdf
# private

MY_TEST_DMP_ID = 16453


In [ ]:
dmpt = DMPTool(DMPTOOL_TOKEN)
stage_dmpt = DMPTool(STAGING_DMPTOOL_TOKEN, prod=False)

In [ ]:
# get counts for various institutions 
# https://via.hypothes.is/https://github.com/CDLUC3/dmptool/wiki/API#for-a-list-of-institutions-and-plans-count
# curl -v -X GET "https://dmptool.org/api/v1/institutions_plans_count"

#plans_counts = requests.get("https://dmptool.org/api/v1/institutions_plans_count").json()

plans_counts = dmpt.institutions_plans_count()

df = DataFrame([r.get('institution') for r in plans_counts])
df.head()

In [ ]:
df.sort_values(by='plans_count', ascending=False).head()

In [ ]:
# list of plans
# curl -H  "Authorization: Token token=token_number" https://dmptool.org/api/v1/plans

#headers = {'Authorization': 'Token token={}'.format(DMPTOOL_TOKEN)}
#r = requests.get("https://dmptool.org/api/v1/plans", headers=headers)
#r.json()[:2]

dmpt.plans()[:2]

In [ ]:
plans_df = DataFrame([r.get('plan') for r in dmpt.plans()])
plans_df.head()

In [ ]:
# thought I'd be able to get a list of my own DMPs -- but no
# only public dmps? https://dmptool.org/public_dmps?public%3Aall_scope=all&public%3Apage=6

np.any(plans_df.owner.apply(lambda o: o.get('email')) == 'yee@berkeley.edu')

In [ ]:
# let's try to get a plan
# curl -H  "Authorization: Token token=token_number" https://dmptool.org/api/v1/plans/:id

headers = {'Authorization': 'Token token={}'.format(DMPTOOL_TOKEN)}
r = requests.get("https://dmptool.org/api/v1/plans/{}".format(MY_TEST_DMP_ID), headers=headers)
r.json()


In [ ]:
dmpt.plans(MY_TEST_DMP_ID)

In [ ]:
# get the json rendition of my private plan

headers = {'Authorization': 'Token token={}'.format(DMPTOOL_TOKEN)}

r = requests.get("https://dmptool.org/api/v1/plans_full/{}".format(MY_TEST_DMP_ID), headers=headers)
r.json()

In [ ]:
dmpt.plans_full(MY_TEST_DMP_ID)

In [ ]:
# add .pdf to retrieve these file formats via the API

headers = {'Authorization': 'Token token={}'.format(DMPTOOL_TOKEN)}

r = requests.get("https://dmptool.org/api/v1/plans_full/{}.pdf".format(MY_TEST_DMP_ID), headers=headers)

In [ ]:
dmpt.plans_full(MY_TEST_DMP_ID, 'pdf')

In [ ]:
dmpt.plans_full(MY_TEST_DMP_ID, 'docx')

In [ ]:
# plans_full

r = requests.get("https://dmptool.org/api/v1/plans_full/")

In [ ]:
len(r.json())

In [ ]:
# a json doc for to represent all public docs

len(dmpt.plans_full())

# Testing of access to private plans (2016.02.25)

[Only return private plans (plans owned by the user associated with the AUTH token). · Issue #164 · CDLUC3/dmptool](https://github.com/CDLUC3/dmptool/issues/164#issuecomment-188744502)

In [ ]:
STAGING_DMP_PLAN_ID = 1534

In [ ]:
# let's try to get a plan
# curl -H  "Authorization: Token token=token_number" https://dmptool.org/api/v1/plans/:id

headers = {'Authorization': 'Token token={}'.format(STAGING_DMPTOOL_TOKEN)}
r = requests.get("https://dmp2-staging.cdlib.org/api/v1/plans/{}".format(STAGING_DMP_PLAN_ID), headers=headers)
r.content

In [ ]:
stage_dmpt.plans(STAGING_DMP_PLAN_ID)

In [ ]:
# can get full private dmp from staging server
headers = {'Authorization': 'Token token={}'.format(STAGING_DMPTOOL_TOKEN)}

r = requests.get("https://dmp2-staging.cdlib.org/api/v1/plans_full/{}".format(STAGING_DMP_PLAN_ID), headers=headers)
r.json()

In [ ]:
stage_dmpt.plans_full(STAGING_DMP_PLAN_ID)

In [ ]:
# plans owned

headers = {'Authorization': 'Token token={}'.format(STAGING_DMPTOOL_TOKEN)}

r = requests.get("https://dmp2-staging.cdlib.org/api/v1/plans_owned", headers=headers)
r.json()

In [ ]:
stage_dmpt.plans_owned()

In [ ]:
# plans owned full

headers = {'Authorization': 'Token token={}'.format(STAGING_DMPTOOL_TOKEN)}

r = requests.get("https://dmp2-staging.cdlib.org/api/v1/plans_owned_full", headers=headers)
r.json()

In [ ]:
stage_dmpt.plans_owned_full()

# Testing of access to private plans (2016.03.16)

[Only return private plans (plans owned by the user associated with the AUTH token). · Issue #164 · CDLUC3/dmptool](https://github.com/CDLUC3/dmptool/issues/164#issuecomment-188744502)


* https://github.com/CDLUC3/dmptool/wiki/API#for-a-specific-plan-with-all-related-attributes
* https://github.com/CDLUC3/dmptool/wiki/API#for-a-list-of-plans-owned-or-co-owned-by-a-user
* https://github.com/CDLUC3/dmptool/wiki/API#for-a-list-of-plans-and-all-related-attributes-owned-or-co-owned-by-a-user



In [ ]:
dmpt.plans_owned()

In [ ]:
dmpt.plans_owned_full()

In [ ]:
dmpt.plans_templates()